In [237]:
%matplotlib inline 

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
import sklearn
import datetime
import array 
import math

#from osgeo import ogr

import shapefile as sf
from matplotlib.patches import Polygon
import mpld3

from pyproj import Proj
from censusgeocode import CensusGeocode
from matplotlib.collections import PatchCollection

In [278]:
# read in the processed data. 
predictions = pd.read_csv('output_files/census_predictions_0926.csv')

In [279]:
predictions['population_density'] = ((predictions['total_pop']).astype(float) 
                                     / ((predictions['land_area']).astype(float) * 3.86102e-7))

In [320]:
#predictions['tree_density'] = predictions['tree_density'] / 3.86102e-7
#predictions['sl_density'] = predictions['sl_density'] / 3.86102e-7
predictions['landarea'] = predictions['land_area'] * 3.86102e-7 # square miles. Fractions...hmm.

In [281]:
predictions['rep_per_cap'] = predictions['rep_per_cap'] * 1000.
predictions['crime_per_cap'] = predictions['crime_per_cap'] * 1000. 
predictions['graffiti_per_cap'] = predictions['graffiti_per_cap'] * 1000. 

In [331]:
#predictions['graffiti_reported_per'] = predictions['graffiti_per']
#predictions.loc[np.where(predictions['count']<22)[0], 'graffiti_reported_per'] = 0.0
#predictions['graffiti_reported_per'] = predictions['graffiti_reported_per'] * 100. 
predictions['graffiti_reported_per'] = predictions['graffiti_per'] * 100.

In [283]:
predictions['combined_labels'] = predictions['combined_labels'] * 100. 
predictions['predicted_labels'] = predictions['predicted_labels'] * 100. 

In [284]:
predictions['white_per'] = predictions['white_per'] * 100. 
predictions['latin_per'] = predictions['latin_per'] * 100. 

In [285]:
predictions['limited_english_households_per'] = predictions['limited_english_households_per'] * 100.

In [330]:
predictions[['total_pop', 'rep_per_cap', 'graffiti_per', 'white_per', 'latin_per', 
                 'median_house_income', 'median_age', 'limited_english_households_per', 
                 'tree_density', 'crime_per_cap', 'ap_count', 'predicted_labels', 'landarea', 
            'graffiti_reported_per', 'combined_labels', 'graffiti_per_cap', 
            'sl_density', 'count', 'population_density']].describe()

,total_pop,rep_per_cap,graffiti_per,white_per,latin_per,median_house_income,median_age,limited_english_households_per,tree_density,crime_per_cap,ap_count,predicted_labels,landarea,graffiti_reported_per,combined_labels,graffiti_per_cap,sl_density,count,population_density
count,835.000000,835.000000,835.000000,835.000000,835.000000,835.000000,835.000000,835.000000,835.000000,835.000000,835.000000,835.000000,835.000000,835.000000,835.000000,835.000000,835.000000,835.000000,835.000000
mean,1647.214371,17.716968,0.130749,48.047883,27.664681,70392.211976,36.747425,7.655691,1354.120933,33.539292,0.396407,25.197507,0.420307,5.218007,22.449121,2.881359,301.286434,21.847904,10788.168203
std,1179.883394,51.413443,0.170677,28.613936,24.980165,34877.355135,9.334050,8.980896,950.087535,348.997197,0.801519,17.918503,1.506375,12.030302,17.979098,8.757547,334.301843,26.014423,8293.533331
min,26.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,12.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023625,0.000000,0.000000,0.000000,0.000000,0.000000,20.557691
25%,980.500000,5.263408,0.000000,22.499528,9.570969,45055.500000,30.250000,1.022383,730.932779,5.333703,0.000000,11.630336,0.088734,0.000000,7.190809,0.000000,154.733918,8.000000,5107.938288
50%,1410.000000,10.139860,0.066667,52.537845,18.147918,64000.000000,35.500000,4.983389,1114.394232,11.331445,0.000000,23.945353,0.155890,0.000000,19.776978,0.723589,235.151986,15.000000,8892.968012
75%,2035.000000,18.469154,0.200000,72.517231,36.345727,89658.000000,41.850000,10.833459,1795.593668,23.253436,1.000000,36.471239,0.289497,4.000000,33.753809,2.470152,343.039617,26.000000,14201.840846
max,16326.000000,1384.615385,1.000000,100.000000,100.000000,250000.000000,82.300000,56.331878,7260.844362,10038.461538,7.000000,82.054054,24.002545,82.608696,82.608696,153.846154,3645.569726,301.000000,59560.590338


        "PREDICTED_": "0.414026738229", 
        "LIMITED_EN": "0.302272727273", 
        "FUNCSTAT": "S", 
        "AWATER": 0, 
        "COUNTYFP": "073", 
        "LATIN_PER": "0.969094922737", 
        "MTFCC": "G5030", 
        "STATEFP": "06", 
        "INTPTLAT": "+32.5564082", 
        "BLKGRPCE": "2", 
        "CRIME_PER_": "0.0", 
        "INTPTLON": "-117.0507980", 
        "MEDIAN_AGE": "29.0", 
        "NAMELSAD": "Block Group 2", 
        "MEDIAN_HOU": "29516.0", 
        "TRACTCE": "010013", 
        "WHITE_PER": "0.0140728476821", 
        "TOTAL_POP": "3624.0", 
        "GEOID": "060730100132", 
        "ALAND": 404606, 
        "AP_COUNT": "0.0", 
        "TREE_DENSI": "0.000402861054952", 
        "REP_PER_CA": "0.0030353200883", 
        "GRAFFITI_P": "0.636363636364"


In [332]:
# read the shapefile
#reader = sf.Reader("shapefiles/sandiego_city_blockgroups.shp")
reader = sf.Reader("shapefiles/sandiego_city_blockgroups_modeladded.shp")

w = sf.Writer()

# Copy over the existing fields
w.fields = list(reader.fields)

#column_values = ['total_pop', 'rep_per_cap', 'graffiti_per', 'white_per', 'latin_per', 
#                 'median_house_income', 'median_age', 'limited_english_households_per', 
#                 'tree_density', 'crime_per_cap', 'ap_count', 'predicted_labels']

i = 'graffiti_reported_per'
#w.field(i.upper(), "D", "40")
w.field(i.upper(), "D", "40")

for j, rec in enumerate(reader.records()): 
    rec.append(predictions.loc[j, i])
    # Add the modified record to the new shapefile 
    w.records.append(rec)

# Copy over the geometry without any changes
w._shapes.extend(reader.shapes())

# Save as a new shapefile (or write over the old one)
w.save("shapefiles/sandiego_city_blockgroups_modeladded.shp") 


Convert shapefile into geoJSON file for displaye

In [333]:
# read the shapefile
reader2 = sf.Reader("shapefiles/sandiego_city_blockgroups_modeladded.shp")

fields = reader2.fields[1:]
field_names = [field[0] for field in fields]
buffer = []
for sr in reader2.shapeRecords():
    atr = dict(zip(field_names, sr.record))
    geom = sr.shape.__geo_interface__
    buffer.append(dict(type="Feature", \
    geometry=geom, properties=atr)) 
   

In [334]:
# write the GeoJSON file
from json import dumps
geojson = open("shapefiles/sandiego_city_blockgroups_modeladded.js", "w")
geojson.write(dumps({"type": "FeatureCollection",
                     "features": buffer}, indent=2) + "\n")
geojson.close()
